## Collect the items

the agents goal is to collect all the items in the environment

### utils

In [1]:
from utils.vectors import Vector2D
from utils.canvas import CanvasWithBorders
from utils.algo_utils import (save_algo, load_algo)
from utils.simulations import (simulate_episode, simulate_random_episode, ppo_result_format)

### environment definition

In [8]:
from typing import Set
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import random as rnd
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiDiscrete
from gymnasium.spaces.utils import flatten, flatten_space
import numpy as np
from IPython.display import clear_output
import math
from ipycanvas import Canvas, hold_canvas

class EnvironmentConfiguration: 
    def __init__(self, n_agents, n_targets, agent_range, spawn_area=100, visible_nbrs=1, visible_targets=1, max_steps=None, cache_size=1):
        # parameters that shouldn't affect the agents' behaviour
        self.n_agents = n_agents
        self.n_targets = n_targets
        self.spawn_area = spawn_area
        self.max_steps = max_steps
        # parameters that affect the agents' behavious
        self.agent_range = agent_range
        # parameters that affect the observation space
        self.visible_nbrs = visible_nbrs
        self.visible_targets = visible_targets
        self.cache_size = cache_size

class KeepTheDistance(MultiAgentEnv):
    canvas = None
    CANVAS_WIDTH, CANVAS_HEIGHT = 300.0, 300.0

    def __init__(self, config: EnvironmentConfiguration):
        assert config.n_agents > config.visible_nbrs

        self.n_agents = config.n_agents
        self.n_targets = config.n_targets
        self.spawn_area = config.spawn_area
        self.max_steps = config.max_steps
        self.agent_range = config.agent_range
        self.visible_nbrs = config.visible_nbrs
        self.visible_targets = config.visible_targets
        self.cache_size = config.cache_size

        self.agents_ids = ['agent-' + str(i) for i in range(self.n_agents)]
        self.agent_colors = {agent: self.rgb_to_hex(rnd.randint(0, 255), rnd.randint(0, 255), rnd.randint(0, 255)) for agent in self.agents_ids}
        self.observation_space = self.observation_space('agent-0')
        self.action_space = self.action_space("")

    def unflatten_observation_space(self, agent):
        direction = Box(low=-1, high=1, shape=(2,1), dtype=np.float32)
        distance = Box(low=-np.inf, high=np.inf, shape=(1,1), dtype=np.float32)

        nbrs = Dict({f"nbr-{i}": Dict({'direction': direction, 'distance': distance}) for i in range(self.visible_nbrs)})
        targets = Dict({f"target-{i}": Dict({'direction': direction, 'distance': distance}) for i in range(self.visible_targets)})

        time_t_obs = Dict({"nbrs": nbrs, "targets": targets})

        return Dict({f"t[-{t}]": time_t_obs for t in range(0, self.cache_size)})

    def observation_space(self, agent):
        return flatten_space(self.unflatten_observation_space(agent))

    def action_space(self, agent):
        direction = Box(low=-1.0, high=1.0, shape=(2,1), dtype=np.float32)
        speed = Box(0.0, 1.0, dtype=np.float32)
        return flatten_space(Tuple([direction, speed]))
    
    def __get_time_t_observation(self, agent):
        nbrs_distance_vectors = [Vector2D.distance_vector(self.agents_pos[agent], self.agents_pos[nbr])  
                            for nbr in self.__get_n_closest_neighbours(agent, self.visible_nbrs)]

        targets_distance_vectors = [Vector2D.distance_vector(self.agents_pos[agent], self.targets_pos[target])  
                            for target in self.__get_n_closest_targets(agent, self.visible_targets)]

        nbrs = {
            f"nbr-{i}": {
                "direction": Vector2D.unit_vector(nbrs_distance_vectors[i]).to_np_array(),
                "distance": np.log(1 + Vector2D.norm(nbrs_distance_vectors[i])) #1 - np.exp(-alpha * x)
            }
            for i in range(len(nbrs_distance_vectors))
        }
    
        targets = {
            f"target-{i}": {
                "direction": Vector2D.unit_vector(targets_distance_vectors[i]).to_np_array(),
                "distance": np.log(1 + Vector2D.norm(targets_distance_vectors[i])) #1 - np.exp(-alpha * x)
            }
            for i in range(len(targets_distance_vectors))
        }
        
        for i in range(len(targets_distance_vectors), self.visible_targets):
            targets[f"target-{i}"] = {
                "direction": np.array([0,0], dtype=np.int32),
                "distance": -1 #1 - np.exp(-alpha * x)
            }

        obs = {
            "nbrs": nbrs,
            "targets": targets
        }

        return obs

    def __get_observation(self, agent):
        if len(self.observation_cache[agent]) == 0:
            self.observation_cache[agent] = [self.__get_time_t_observation(agent)]*self.cache_size
        else:
            self.observation_cache[agent] = [self.__get_time_t_observation(agent)] + self.observation_cache[agent]
            self.observation_cache[agent].pop()

        obs = {
            f"t[-{t}]": self.observation_cache[agent][t]
            for t in range(0, self.cache_size)
        }

        return flatten(self.unflatten_observation_space(agent), obs)

    def rgb_to_hex(self, r, g, b):
        return f'#{r:02x}{g:02x}{b:02x}'

    def __get_local_reward(self, agent, action):
        # reward_1: small bonus if the agent collects an item
        reward_1 = +5 if agent in self.collectors else 0

        # reward_2: malus if the agent collides with another agent 
        reward_2= sum([-2 if Vector2D.distance(self.agents_pos[agent], self.agents_pos[nbr]) < self.agent_range else 0 for nbr in self.__get_other_agents(agent)])

        # reward_3: -1 at each step
        reward_3 = -1

        # reward_4: positive reward if the agent moves toward the closest targets, negative otherwise
        distance_diff = ([Vector2D.distance(self.agent_old_pos[agent], self.targets_pos[target]) -
                    Vector2D.distance(self.agents_pos[agent], self.targets_pos[target])
            for target in self.closest_targets[agent]])
        
        reward_4 = max(distance_diff) if len(distance_diff) > 0 else 0

        self.info[agent] = {"info": {f"r2: {reward_2}, r3: {reward_4}, r4: {reward_4}"}}
        return  reward_2 + reward_3 + reward_4*3

    def __get_global_reward(self):
        return self.global_reward * 100
    
    def __get_other_agents(self, agent):
        return [other for other in self.agents_ids if other != agent]

    def __get_n_closest_neighbours(self, agent, n=1):
        distances = {other: Vector2D.distance(self.agents_pos[agent], self.agents_pos[other]) for other in self.__get_other_agents(agent)}
        return [neighbour[0] for neighbour in sorted(list(distances.items()), key=lambda d: d[1])[:n]]
        # return {neighbour[0]: neighbour[1] for neighbour in sorted(list(dst.items()), key=lambda d: d[0])[:n]}

    def __get_n_closest_targets(self, agent, n=1):
        n = min(n, len(self.targets_pos.keys()))
        distances = {target: Vector2D.distance(self.agents_pos[agent], pos) for target, pos in self.targets_pos.items()}
        self.closest_targets[agent] = [target[0] for target in sorted(list(distances.items()), key=lambda d: d[1])[:n]]
        return self.closest_targets[agent]

    def __update_agent_position(self, agent, action):
        unit_movement = Vector2D(action[0], action[1])
        self.agent_old_pos[agent] = self.agents_pos[agent]
        self.agents_pos[agent] = Vector2D.sum(self.agents_pos[agent], Vector2D.mul(unit_movement, action[2]))

    def __collect_items(self):
        self.collectors = []
        uncollected_targets = {}
        for target, target_pos in self.targets_pos.items():
            collected = False
            for agent in self.agents_pos.values():
                if Vector2D.distance(target_pos, agent) < self.agent_range:
                    collected = True
                    self.collectors.append(agent)
            if not collected:
                uncollected_targets[target] = target_pos
        self.targets_pos = uncollected_targets

    def __collect_items_and_compute_global_reward(self):
        old_uncollected_items = len(self.targets_pos.keys())
        self.__collect_items()
        updated_uncollected_items = len(self.targets_pos.keys())
        self.global_reward = old_uncollected_items - updated_uncollected_items

    def reset(self, seed=None, options=None):
        self.steps = 0
        self.agents_pos = {agent: Vector2D.get_random_point(max_x=self.spawn_area, max_y=self.spawn_area) for agent in self.agents_ids}
        self.agent_old_pos = dict(self.agents_pos)
        self.targets_pos = {f"target-{i}": Vector2D.get_random_point(max_x=self.spawn_area, max_y=self.spawn_area) for i in range(self.n_targets)}
        self.collectors = []
        self.closest_targets = {}
        self.info = {}
        self.observation_cache = {agent: [] for agent in self.agents_ids}
        return {agent: self.__get_observation(agent) for agent in self.agents_ids}, {}
     
    def step(self, actions):
        self.steps += 1
        observations, rewards, terminated, truncated, infos = {}, {}, {}, {}, {}

        for agent, action in actions.items():
            self.__update_agent_position(agent, action)

        self.__collect_items_and_compute_global_reward()

        for agent, action in actions.items():
            observations[agent] = self.__get_observation(agent)
            rewards[agent] = self.__get_local_reward(agent, action) + self.__get_global_reward()
            terminated[agent] = False
            truncated[agent] = False
            infos[agent] = self.info[agent]

        truncated['__all__'] = False
        if len(self.targets_pos.keys()) == 0:
            terminated['__all__'] = True
        if self.max_steps != None and self.steps == self.max_steps:
            terminated['__all__'] = True
        else:
            terminated['__all__'] = False

        return observations, rewards, terminated, truncated, infos
     
    def rgb_to_hex(self, r, g, b):
        return f'#{r:02x}{g:02x}{b:02x}'

    def render(self):
        pass

    def get_agent_ids(self):
       return self.agents


class RenderableKeepTheDistance(KeepTheDistance):
    def render(self):
        if self.canvas is None:
            self.canvas = CanvasWithBorders(width=self.CANVAS_WIDTH, height=self.CANVAS_HEIGHT)
            display(self.canvas)
        
        with hold_canvas():
            unit = self.CANVAS_WIDTH/float(self.spawn_area)
            agent_render_size = max(unit,1)
            agent_range_render_size = unit*self.agent_range
            top_left, bottom_right = (0.0,0.0), (self.spawn_area, self.spawn_area)
            self.canvas.clear()

            self.canvas.fill_style = "red"
            for target in self.targets_pos.values():
                raw_pos = target.to_np_array()

                target_pos_in_frame = [((raw_pos[0]-top_left[0])/(bottom_right[0]-top_left[0]))*self.CANVAS_WIDTH,
                        ((raw_pos[1]-top_left[1])/(bottom_right[1]-top_left[1]))*self.CANVAS_HEIGHT,]
                
                self.canvas.fill_circle(
                    target_pos_in_frame[0],
                    target_pos_in_frame[1],
                    1
                )

            for agent in self.agents_ids:
                raw_pos = self.agents_pos[agent].to_np_array()
                color = self.agent_colors[agent]
                
                agent_pos_in_frame = [((raw_pos[0]-top_left[0])/(bottom_right[0]-top_left[0]))*self.CANVAS_WIDTH,
                            ((raw_pos[1]-top_left[1])/(bottom_right[1]-top_left[1]))*self.CANVAS_HEIGHT,]

                self.canvas.fill_style = color
                self.canvas.fill_circle(
                    agent_pos_in_frame[0],
                    agent_pos_in_frame[1],
                    agent_render_size/2.0
                )
                
                self.canvas.stroke_style = "black"
                self.canvas.stroke_circle(
                    agent_pos_in_frame[0],
                    agent_pos_in_frame[1],
                    agent_render_size/2.0
                )

                self.canvas.stroke_style = "red"
                self.canvas.stroke_circle(
                    agent_pos_in_frame[0],
                    agent_pos_in_frame[1],
                    agent_range_render_size
                )

In [7]:
env_config = EnvironmentConfiguration(
    n_agents = 4,
    n_targets = 2,
    spawn_area = 100,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3,
    cache_size=3)

env = RenderableKeepTheDistance(env_config)

print(env.reset()[0]['agent-0'])
env.render()
simulate_random_episode(env, 100, sleep_between_frames=0.03, print_info=False)
#env.step({'agent-1': (1,1,1)})

[ 0.99991345 -0.01315676  4.3438907   0.9957851   0.09171705  4.3479743
  0.83570546 -0.5491779   4.439842    0.7808688  -0.62469506  3.6742415
  0.2631174  -0.9647638   4.060594    0.          0.         -1.
  0.99991345 -0.01315676  4.3438907   0.9957851   0.09171705  4.3479743
  0.83570546 -0.5491779   4.439842    0.7808688  -0.62469506  3.6742415
  0.2631174  -0.9647638   4.060594    0.          0.         -1.
  0.99991345 -0.01315676  4.3438907   0.9957851   0.09171705  4.3479743
  0.83570546 -0.5491779   4.439842    0.7808688  -0.62469506  3.6742415
  0.2631174  -0.9647638   4.060594    0.          0.         -1.        ]


CanvasWithBorders(height=300, width=300)

-0.0419645466054952
0.7205040169434582
0.05460370904376788
-0.26290442748749143
-0.30536286000073787
0.08478925934286252
0.1818537425983422
0.42006796375359556
0.08425288128137254
-0.07742643100480606
0.11212917881842799
0.6387183065614153
-0.23030367551744746
-0.43747122839783614
0.23745495419001372
0.15847057058822855
-0.27710592470358364
-0.2511834129640178
0.22827457109975313
0.2993045229891962
0.20868433148500998
-0.28684835993841773
0.2118994013233575
-0.0028165990922843775
0.3231193816745339
0.15642170327328486
0.016506790921514636
0.28418283699910774
0.35098751217441304
-0.14934726651630115
0.13513815138038154
0.13359108758311322
0.24069645001193862
-0.1630173257387355
0.06325507467997227
0.13554715500762882
-0.175861156352763
0.14308738973121393
0.17709433001532737
0.03789413259124785
0.524239614272993
0.14746954655723243
0.525759370178017
-0.05347061894926952
-0.37956541360583174
-0.5501720195463093
0.2516971673582269
0.03717497891383914
-0.05723010260259187
0.076804984006336

## policy training

In [32]:
import ray
ray.shutdown()

## collect_the_items?visible_nbrs=3&visible_targets=3

In [9]:
from ray.tune.registry import register_env

env_config = EnvironmentConfiguration(
    n_agents = 4,
    n_targets = 10,
    spawn_area = 100,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3)
register_env("collect_the_items?visible_nbrs=3&visible_targets=3", lambda _: KeepTheDistance(env_config))

In [9]:
algo = load_algo("collect_the_items?visible_nbrs=3&visible_targets=3")

2024-05-24 16:05:18,459	WARNING deprecation.py:50 -- DeprecationWarning: `_enable_new_api_stack` has been deprecated. Use `AlgorithmConfig._enable_new_api_stack` instead. This will raise an error in the future!
/home/nicolo/anaconda3/envs/tianEnv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/nicolo/anaconda3/envs/tianEnv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be r

In [10]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

from gymnasium.wrappers.time_limit import TimeLimit

trainin_steps = 60

algo = (
    PPOConfig()
    .training(gamma = 0.95, 
              lr = 0.001,
              train_batch_size = 4096, 
              sgd_minibatch_size = 256, 
              num_sgd_iter = 30,
              #entropy_coeff=0.005,
              )
    .env_runners(num_env_runners=1)
    .resources(num_gpus=0)
    .environment(env="collect_the_items?visible_nbrs=3&visible_targets=3")
    .build()
)
clear_output()

out = ""
for i in range(trainin_steps):
    result = algo.train()
    clear_output()
    out += ppo_result_format(result) + "\n"
    print(out)
    simulate_episode(RenderableKeepTheDistance(env_config), algo, 500, sleep_between_frames=0.01, print_reward=True, print_info=True)

iteration [1] => episode_reward_mean: -35.76409825378267, episode_len_mean: 300.0, agent_steps_trained: 16384, env_steps_trained: 4096, entropy: 4.273375653599699, learning_rate: 0.0010000000000000005
iteration [2] => episode_reward_mean: 288.2868533364462, episode_len_mean: 300.0, agent_steps_trained: 32768, env_steps_trained: 8192, entropy: 4.258766336242358, learning_rate: 0.0010000000000000005
iteration [3] => episode_reward_mean: 411.98204586615265, episode_len_mean: 300.0, agent_steps_trained: 49152, env_steps_trained: 12288, entropy: 4.230659523357947, learning_rate: 0.0010000000000000005
iteration [4] => episode_reward_mean: 598.2203885673239, episode_len_mean: 300.0, agent_steps_trained: 65536, env_steps_trained: 16384, entropy: 4.204704008251428, learning_rate: 0.0010000000000000005
iteration [5] => episode_reward_mean: 753.1944001465281, episode_len_mean: 300.0, agent_steps_trained: 81920, env_steps_trained: 20480, entropy: 4.184859064593911, learning_rate: 0.001000000000000

CanvasWithBorders(height=300, width=300)

info:  {'agent-0': {'info': {'r2: 0, r3: -0.3014581584779492, r4: -0.3014581584779492'}}, 'agent-1': {'info': {'r2: 0, r3: 0.30457233201897793, r4: 0.30457233201897793'}}, 'agent-2': {'info': {'r2: 0, r3: 0.47131337606327506, r4: 0.47131337606327506'}}, 'agent-3': {'info': {'r2: 0, r3: 0.07827115016064212, r4: 0.07827115016064212'}}}
reward:  {'agent-0': -1.9043744754338476, 'agent-1': -0.0862830039430662, 'agent-2': 0.4139401281898252, 'agent-3': -0.7651865495180736} 

info:  {'agent-0': {'info': {'r2: 0, r3: 0.21286945573640637, r4: 0.21286945573640637'}}, 'agent-1': {'info': {'r2: 0, r3: 0.1027255498198727, r4: 0.1027255498198727'}}, 'agent-2': {'info': {'r2: 0, r3: 0.35279506229162827, r4: 0.35279506229162827'}}, 'agent-3': {'info': {'r2: 0, r3: 0.0, r4: 0.0'}}}
reward:  {'agent-0': -0.3613916327907809, 'agent-1': -0.6918233505403819, 'agent-2': 0.05838518687488481, 'agent-3': -1.0} 

info:  {'agent-0': {'info': {'r2: 0, r3: 1.2310917468550677, r4: 1.2310917468550677'}}, 'agent-1':

KeyboardInterrupt: 

In [108]:
env_config_2 = EnvironmentConfiguration(
    n_agents = 10,
    n_targets = 10,
    spawn_area = 500,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3)
simulate_episode(RenderableKeepTheDistance(env_config_2), algo, 300, sleep_between_frames=0.01, print_info=False)

CanvasWithBorders(height=300, width=300)

In [100]:
save_algo(algo, "collect_the_items?visible_nbrs=3&visible_targets=3")

An Algorithm checkpoint has been created inside directory: 'TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/mnt/c/Users/nicol/Desktop/Università/tesi/experiments/RL_experiments/algos/collect_the_items?visible_nbrs=3&visible_targets=3), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 22.84449312289556, 'cur_kl_coeff': 38.9239013671875, 'cur_lr': 0.0010000000000000005, 'total_loss': 6.04471684663246, 'policy_loss': -0.0011021292180278881, 'vf_loss': 5.8489303250486655, 'vf_explained_var': 0.031994151324033736, 'kl': 0.0050582971823511635, 'entropy': 3.3903824771444, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 256.0, 'num_grad_updates_lifetime': 114240.5, 'diff_num_grad_updates_vs_sampler_policy': 959.5}}, 'num_env_steps_sampled': 245760, 'num_env_steps_trained': 245760, 'num_agent_steps_sampled': 983040, 'num_agent_steps_trained': 983

## collect_the_items?visible_nbrs=3&visible_targets=3&cache_size=3

In [142]:
from ray.tune.registry import register_env

env_config = EnvironmentConfiguration(
    n_agents = 4,
    n_targets = 10,
    spawn_area = 100,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3,
    cache_size=2)
register_env("collect_the_items?visible_nbrs=3&visible_targets=3&cache_size=3", lambda _: KeepTheDistance(env_config))

In [ ]:
algo = load_algo("collect_the_items?visible_nbrs=3&visible_targets=3&cache_size=3")

In [144]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

from gymnasium.wrappers.time_limit import TimeLimit

trainin_steps = 60

algo = (
    PPOConfig()
    .training(gamma = 0.925, 
              lr = 0.001,
              train_batch_size = 4096, 
              sgd_minibatch_size = 256, 
              num_sgd_iter = 30,
              #entropy_coeff=0.005,
              )
    .env_runners(num_env_runners=1)
    .resources(num_gpus=0)
    .environment(env="collect_the_items?visible_nbrs=3&visible_targets=3&cache_size=3")
    .build()
)
clear_output()

out = ""
for i in range(trainin_steps):
    result = algo.train()
    clear_output()
    out += ppo_result_format(result) + "\n"
    print(out)
    simulate_episode(RenderableKeepTheDistance(env_config), algo, 500, sleep_between_frames=0.01, print_reward=True)

iteration [1] => episode_reward_mean: -802.1538461538462, episode_len_mean: 300.0, agent_steps_trained: 16384, env_steps_trained: 4096, entropy: 4.285082547614972, learning_rate: 0.0010000000000000005
iteration [2] => episode_reward_mean: -636.4444444444445, episode_len_mean: 300.0, agent_steps_trained: 32768, env_steps_trained: 8192, entropy: 4.257554467519125, learning_rate: 0.0010000000000000005
iteration [3] => episode_reward_mean: -529.6, episode_len_mean: 300.0, agent_steps_trained: 49152, env_steps_trained: 12288, entropy: 4.23101422910889, learning_rate: 0.0010000000000000005
iteration [4] => episode_reward_mean: -482.8888888888889, episode_len_mean: 300.0, agent_steps_trained: 65536, env_steps_trained: 16384, entropy: 4.211818875248233, learning_rate: 0.0010000000000000005
iteration [5] => episode_reward_mean: -454.6470588235294, episode_len_mean: 300.0, agent_steps_trained: 81920, env_steps_trained: 20480, entropy: 4.184312181919813, learning_rate: 0.0010000000000000005
itera

CanvasWithBorders(height=300, width=300)

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': 99, 'agent-1': 99, 'agent-2': 99, 'agent-3': 99} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3': -1} 

reward:  {'agent-0': -1, 'agent-1': -1, 'agent-2': -1, 'agent-3'

KeyboardInterrupt: 

In [146]:
env_config_2 = EnvironmentConfiguration(
    n_agents = 10,
    n_targets = 10,
    spawn_area = 500,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3,
    cache_size=2)
simulate_episode(RenderableKeepTheDistance(env_config_2), algo, 300, sleep_between_frames=0.01, print_info=False)

CanvasWithBorders(height=300, width=300)

In [ ]:
save_algo(algo, "collect_the_items?visible_nbrs=3&visible_targets=3&cache_size=3")